## Outline of this code

The function of this code is to create a database suited for the purposes of this project from the data [provided by Wyatt Walsh on kaggle](https://www.kaggle.com/datasets/wyattowalsh/basketball) 

In [1]:
###     Dependencies



## Reflect original NBA database (did not work)

In [57]:
#   Dependencies: SQLAlchemy
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [67]:
#   Create engine to nba.sqlite
engine = create_engine("sqlite:///../nba.sqlite")

#   Reflect existing database into a new model
Base = automap_base()

#   Reflect the tables
Base.prepare(autoload_with = engine)

In [69]:
# View all the classes automap found
Base.classes.keys()

[]

This is an unexpected error –– [this stack overflow forum](https://stackoverflow.com/questions/42946174/sqlalchemy-automap-not-generating-base-classes-table-name) discusses a potential issue with `automap_base()` not working with sqlite tables without primary keys. The Kaggle dataset also provides a csv, so I will manually set up a db instead.

# Database preparation

## Database Modelling

#### Draft 1

![alt](QuickDBD-export.png)

The above is the ERD I sketched based on the CSV files I was provided.

- kept variables of interest

- This proved to be quite out of scope for the task at hand (involving sqlalchemy and sqlite) –– A simpler single table will be used

#### Draft 2

![alt](QuickDBD-export2.png)

This single table is better suited for conveniently faciliating the analysis.

- note the underscore and (imported_from_______) "columns"; these are just to denote that these columns have to be joined.

## CSV Creation

In [6]:
#   Dependencies
import pandas as pd


In [7]:
#   Read in CSVs
team_details_df = pd.read_csv("./../csv/team_details.csv")
game_info_df = pd.read_csv("./../csv/game_info.csv")
game_df = pd.read_csv("./../csv/game.csv")

In [12]:
#   Select only columns of interest:
team_details_df = team_details_df[["team_id", "arena"]]
game_info_df = game_info_df[["game_id", "attendance"]]
game_df = game_df[["game_id", "game_date", "team_id_home", "team_id_away", "team_name_home", "team_abbreviation_home", "team_name_away", "team_abbreviation_away",
                     "matchup_home", "wl_home", "wl_away", 
                     "fgm_home", "fg3m_home", "fga_home", "fg_pct_home", "ftm_home", "fta_home", "ft_pct_home","ast_home","stl_home","blk_home", "tov_home", "pts_home",
                     "fgm_away", "fg3m_away", "fga_away", "fg_pct_away", "ftm_away", "fta_away", "ft_pct_away","ast_away","stl_away","blk_away","tov_away", "pts_away"]]

# Retroactively added ast, stl, blk, tov, fg3m(written for record keeping)

In [15]:
#   Create fg2m columns
game_df["fg2m_home"] = game_df["fgm_home"] - game_df["fg3m_home"]
game_df["fg2m_away"] = game_df["fgm_away"] - game_df["fg3m_away"]

game_df[["fgm_home","fg2m_home","fg3m_home","fgm_away","fg2m_away","fg3m_away"]].tail()

,fgm_home,fg2m_home,fg3m_home,fgm_away,fg2m_away,fg3m_away
62356,41.0,31.0,10.0,39.0,24.0,15.0
62357,41.0,25.0,16.0,44.0,29.0,15.0
62358,34.0,22.0,12.0,37.0,28.0,9.0
62359,46.0,23.0,23.0,43.0,22.0,21.0
62360,44.0,31.0,13.0,41.0,26.0,15.0


##### Manually create latitude and longitude columns tfor team_details_df

In [16]:
#   View df
coords =[
    [33.757222, -84.916944],
    [42.366303,-71.062228],
    [41.496389, -81.688056],
    [29.948889, -90.081944],
    [41.880556, -87.674167],
    [32.790556, -96.810278],    
    [39.748611, -105.0075],
    [37.768056, -122.3875],
    [29.750833, -95.362222],
    [34.043056, -118.267222],
    [34.043056, -118.267222],
    [25.781389, -80.188056],
    [43.043611, -87.916944],
    [44.979444, -93.276111],
    [40.68265, -73.974689],
    [40.750556, -73.993611],
    [28.539167, -81.383611],
    [39.763889, -86.155556],
    [39.901111, -75.171944],
    [33.445833, -112.071389],
    [45.531667, -122.666667],
    [38.649167, -121.518056],
    [29.426944, -98.4375],
    [35.463333, -97.515],
    [43.643333, -79.379167],
    [40.768333, -111.901111],
    [35.138333, -90.050556],
    [38.898056, -77.020833],
    [42.341111, -83.055],
    [35.225, -80.839167]
]

lat = [x[0] for x in coords]
lon = [x[1] for x in coords]


team_details_df["arena_lat"] = lat
team_details_df["arena_lon"] = lon

team_details_df.head()

,team_id_home,arena,arena_lat,arena_lon
0,1610612737,State Farm Arena,33.757222,-84.916944
1,1610612738,TD Garden,42.366303,-71.062228
2,1610612739,Rocket Mortgage FieldHouse,41.496389,-81.688056
3,1610612740,Smoothie King Center,29.948889,-90.081944
4,1610612741,United Center,41.880556,-87.674167


----

In [17]:
#   Merge the dataframes into a single df
new_game_df = pd.merge(game_df, game_info_df, how = "left", on = "game_id")

#   The purpose of merging the team_details_df on the other df's home id is because it is assumed that the arena in which the match takes place is in the home arena
team_details_df.rename(columns = {"team_id": "team_id_home"}, inplace = True)

new_game_df = pd.merge(new_game_df, team_details_df, how = "left", on = "team_id_home")

# Reorder columns

# add new ones: ast, stl, blk, tov, fg3m, and fg2m (new column count should go from 29 to 41)
new_game_df = new_game_df[["game_id", "game_date", "arena","arena_lat", "arena_lon", "attendance", "team_id_home", "team_id_away", "team_name_home", "team_abbreviation_home", "team_name_away", "team_abbreviation_away",
                     "matchup_home", "wl_home", "wl_away", "fgm_home", "fg2m_home", "fg3m_home", "fga_home", "fg_pct_home", "ftm_home", "fta_home", "ft_pct_home","ast_home","stl_home","blk_home","tov_home", "pts_home", 
                     "fgm_away","fg2m_away", "fg3m_away", "fga_away", "fg_pct_away", "ftm_away", "fta_away", "ft_pct_away","ast_away","stl_away","blk_away","tov_away", "pts_away"]]



#   Drop NA
new_game_df.dropna(inplace = True)

new_game_df.head()



,game_id,game_date,arena,arena_lat,arena_lon,attendance,team_id_home,team_id_away,team_name_home,team_abbreviation_home,...,fga_away,fg_pct_away,ftm_away,fta_away,ft_pct_away,ast_away,stl_away,blk_away,tov_away,pts_away
4725,26000179,1961-01-15 00:00:00,Wells Fargo Center,39.901111,-75.171944,2694.0,1610612755,1610612744,Syracuse Nationals,SYR,...,96.0,0.458,25.0,35.0,0.714,22.0,0.0,0.0,0.0,113
4742,26000193,1961-01-22 00:00:00,Wells Fargo Center,39.901111,-75.171944,4260.0,1610612755,1610612747,Syracuse Nationals,SYR,...,97.0,0.392,33.0,41.0,0.805,14.0,0.0,0.0,0.0,109
4751,26000203,1961-01-27 00:00:00,Golden 1 Center,38.649167,-121.518056,5177.0,1610612758,1610612755,Cincinnati Royals,CIN,...,114.0,0.500,24.0,34.0,0.706,0.0,0.0,0.0,0.0,138
4795,26000246,1961-02-12 00:00:00,Wells Fargo Center,39.901111,-75.171944,2734.0,1610612755,1610612765,Syracuse Nationals,SYR,...,108.0,0.444,26.0,34.0,0.765,25.0,0.0,0.0,0.0,122
17582,28100068,1981-11-10 00:00:00,Little Caesars Arena,42.341111,-83.055000,10314.0,1610612765,1610612755,Detroit Pistons,DET,...,88.0,0.443,17.0,24.0,0.708,26.0,13.0,14.0,24.0,95


In [18]:
#   Export data frame as csv
new_game_df.to_csv("./csv/nba_games.csv", index = False)

---

## Database Creation

In [19]:
#   Dependencies
import pandas as pd

# SQLAlchemy
import sqlalchemy
from sqlalchemy import create_engine, inspect
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
from sqlalchemy import Column, Integer, String, Float, DateTime

In [20]:
#   Read in csv
games = pd.read_csv("./csv/nba_games.csv")

#   Set game_id as the index
games.set_index("game_id", inplace = True)

#Preview
games.head()

,game_date,arena,arena_lat,arena_lon,attendance,team_id_home,team_id_away,team_name_home,team_abbreviation_home,team_name_away,...,fga_away,fg_pct_away,ftm_away,fta_away,ft_pct_away,ast_away,stl_away,blk_away,tov_away,pts_away
game_id,,,,,,,,,,,,,,,,,,,,,
26000179,1961-01-15 00:00:00,Wells Fargo Center,39.901111,-75.171944,2694.0,1610612755,1610612744,Syracuse Nationals,SYR,Philadelphia Warriors,...,96.0,0.458,25.0,35.0,0.714,22.0,0.0,0.0,0.0,113
26000193,1961-01-22 00:00:00,Wells Fargo Center,39.901111,-75.171944,4260.0,1610612755,1610612747,Syracuse Nationals,SYR,Los Angeles Lakers,...,97.0,0.392,33.0,41.0,0.805,14.0,0.0,0.0,0.0,109
26000203,1961-01-27 00:00:00,Golden 1 Center,38.649167,-121.518056,5177.0,1610612758,1610612755,Cincinnati Royals,CIN,Syracuse Nationals,...,114.0,0.500,24.0,34.0,0.706,0.0,0.0,0.0,0.0,138
26000246,1961-02-12 00:00:00,Wells Fargo Center,39.901111,-75.171944,2734.0,1610612755,1610612765,Syracuse Nationals,SYR,Detroit Pistons,...,108.0,0.444,26.0,34.0,0.765,25.0,0.0,0.0,0.0,122
28100068,1981-11-10 00:00:00,Little Caesars Arena,42.341111,-83.055000,10314.0,1610612765,1610612755,Detroit Pistons,DET,Philadelphia 76ers,...,88.0,0.443,17.0,24.0,0.708,26.0,13.0,14.0,24.0,95


In [21]:
#   Establish Base for table's class construction
Base = declarative_base()

# games table class construction
class Games(Base):
    __tablename__ = "games"

    game_id = Column(Integer, primary_key = True)
    game_date = Column(DateTime)
    arena = Column(String)
    arena_lat = Column(Float)
    arena_lon = Column(Float)
    attendance = Column(Integer)
    team_id_home = Column(Integer) 
    team_id_away = Column(Integer)
    team_name_home = Column(String)
    team_abbreviation_home = Column(String)
    team_name_away = Column(String)
    team_abbreviation_away = Column(String)
    matchup_home = Column(String)
    wl_home = Column(String)
    wl_away = Column(String)
    fgm_home = Column(Integer)
    fg2m_home = Column(Integer)
    fg3m_home = Column(Integer)
    fga_home = Column(Integer)
    fg_pct_home = Column(Integer)
    ftm_home = Column(Integer)
    fta_home = Column(Integer)
    ft_pct_home = Column(Integer)
    ast_home = Column(Integer)
    stl_home = Column(Integer)
    blk_home = Column(Integer)
    tov_home = Column(Integer)
    pts_home = Column(Integer)
    fgm_away = Column(Integer)
    fg2m_away = Column(Integer)
    fg3m_away = Column(Integer)
    fga_away = Column(Integer)
    fg_pct_away = Column(Integer)
    ftm_away = Column(Integer)
    fta_away = Column(Integer)
    ft_pct_away = Column(Integer)
    ast_away = Column(Integer)
    stl_away = Column(Integer)
    blk_away = Column(Integer)
    tov_away = Column(Integer)
    pts_away = Column(Integer)

In [22]:
#   Create engine
engine = create_engine("sqlite:///NBA.sqlite")

#   Connect to database
con = engine.connect()

#   Create the games table within the database
Base.metadata.create_all(con)

#    Import CSV into the database table
games.to_sql("games", con, if_exists = "append")


- if an error occurs during the above cell, consider whether it is due to the last method and the argument `if_exists`.

    - An existing database will not be able to append duplicate data, so consider a method of 'updating it'

        -   I would simply delete the table and use `declarative_base()` again, but I haven't tested it for updating

#